In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pathlib import Path

import dill as pickle
import numpy as np
import plotly.graph_objects as go
import scipy.io
import visu3d as v3d

from burybarrel.plotting import get_surface_line_traces, get_ray_trace
from burybarrel.synthbarrel import Cylinder

In [ ]:
reconstr_dir = Path("../results/barrel4-2sec-first5-reconstr")
# reconstr_dir = Path("../results/barrelddt1-reconstr")
# reconstr_dir = Path("../results/dive8-barrel-15-04-2sec-middle-reconstr")
# reconstr_dir = Path("../results/barrel2-5sec-contrast-reconstr")
ptspath = reconstr_dir / "rotatedpts.pickle"
matcylpath = reconstr_dir / "matcyl.mat"

if not ptspath.exists() or not matcylpath.exists():
    raise FileNotFoundError("sad :(")

with open(ptspath, "rb") as f:
    rotpcdict = pickle.load(f)
matcylparams = scipy.io.loadmat(matcylpath)["cylparams"][0]

rotatedpc = v3d.Point3d(p=rotpcdict["p"], rgb=rotpcdict["rgb"])
barrelyes = rotpcdict["isbarrel"]
cyl = Cylinder.from_mat_params(matcylparams)
upax = cyl.axis
if upax[2] < 0:
    upax = -upax

In [ ]:
print(cyl.get_volume_ratio_monte(100000, planecoeffs=[0, 0, 1, 0]))

xx, yy, zz = cyl.get_pts_surf()
# need to do this to change scatter point size
randidxs = np.random.choice(np.arange(rotatedpc.shape[0]), 40000, replace=False)
randmask = np.zeros(rotatedpc.shape[0], dtype=bool)
randmask[randidxs] = True
pctrace = rotatedpc[(~barrelyes) & randmask].make_traces()[0]
pctrace["marker"]["size"] = 4.0
pctrace["opacity"] = 0.9
pctracebarrel = rotatedpc[barrelyes & randmask].make_traces()[0]
pctracebarrel["marker"]["size"] = 4.0
pctracebarrel["opacity"] = 0.9
fig = v3d.make_fig()
fig.add_traces([pctrace, pctracebarrel])
fig.add_trace(get_ray_trace(cyl.c, upax, length=cyl.h, width=8, color="#e81b00", markersize=10))
cylsurf = go.Surface(x=xx, y=yy, z=zz, opacity=0.4, surfacecolor=np.zeros_like(xx), colorscale="oranges")
fig.add_trace(cylsurf)
fig.add_traces(get_surface_line_traces(xx, yy, zz, width=2, step=5, include_horizontal=False))
xmin, xmax = np.min(rotatedpc.p[:, 0]), np.max(rotatedpc.p[:, 0])
ymin, ymax = np.min(rotatedpc.p[:, 1]), np.max(rotatedpc.p[:, 1])
xx, yy = np.meshgrid(np.linspace(xmin, xmax, 10), np.linspace(ymin, ymax, 10))
zz = np.zeros_like(xx)
planesurf = go.Surface(x=xx, y=yy, z=zz, opacity=0.1, colorscale="purples")
# fig.add_trace(planesurf)
# fig.add_traces(get_surface_line_traces(xx, yy, zz))
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.show()